In [2]:
# Loading necessary libraries


import opensoundscape
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import librosa
import torch
import random
import sys
from PIL import Image
from scipy.ndimage import median_filter
from sklearn.preprocessing import StandardScaler
from PIL import Image as im 
from pathlib import Path
import cv2


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\opensoundscape\ml\cnn.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# Load Datasets

annotations = pd.read_csv("../raw_sonobuoy_images/modified_annotations.csv")

unique_annotation = annotations.drop_duplicates(subset=['spectrogram_path'])

annotations_modded = annotations.copy()

annotations_modded["spectrogram_path"] = annotations["spectrogram_path"].str.replace('raw_sonobuoy_images', 'processed_sonobuoy_images', regex=False)

annotations_modded.to_csv("../processed_sonobuoy_images/modified_annotations.csv", index=False)



In [4]:
annotations.head()

,spectrogram_path,label,xmin,ymin,xmax,ymax
0,../raw_sonobuoy_images/CC0407-SB10-040717-2222...,D,406,83,447,123
1,../raw_sonobuoy_images/CC0407-SB10-040717-2222...,D,106,83,146,123
2,../raw_sonobuoy_images/CC0407-SB17-040719-2050...,D,431,67,471,123
3,../raw_sonobuoy_images/CC0407-SB17-040719-2050...,D,131,67,171,123
4,../raw_sonobuoy_images/CC0407-SB2-040713-23500...,D,338,77,372,129


In [5]:
# Defining draw image functions

def draw_img(ax, img_vector, h=141, w=601):
    """
    1. takes img_vector,
    2. reshapes into right dimensions,
    3. draws the resulting image
    """
    
    
    ax.imshow( (img_vector).reshape(h,w), cmap=plt.cm.gray)
    
    plt.xticks(())
    plt.yticks(())

def draw_img_single(img_vector, h=141, w=601):
    """
    1. takes img_vector,
    2. reshapes into right dimensions,
    3. draws the resulting image
    """
    
    
    plt.imshow( (img_vector).reshape(h,w), cmap=plt.cm.gray)
    
    plt.xticks(())
    plt.yticks(())

In [6]:
# Here we construct the matrix of image data (rows are spectrograms and columns are pixel values for each spectrogram)

data_matrix = []

for index, row in unique_annotation.iterrows():

    image = Image.open(row['spectrogram_path'])

    pixel_values = np.array(list(image.getdata()))

    data_matrix.append(pixel_values)

stacked_specs = np.vstack(data_matrix)


scaler = StandardScaler(with_std=False)
data_matrix_mod1 = scaler.fit_transform(stacked_specs)
original_data = data_matrix_mod1

In [11]:

# Singular Value Decomposition


# U is the eigenbasis of the original space (eigen vectors span directions of maximal variance)

# S is the diagonal matrix of singular values of "original_data" that determine the extent to which the eigen axes are stretched to create the original features; these are the square roots of the eigenvalues for the covariance matrix of the data

# T is the transpose of the matrix whose columns represent the principal components of "original_data" (these columns are the orthonormal basis vectors of the transformed space that point in the directions of maximal variance)

U, S, T = np.linalg.svd(original_data, full_matrices=False)

US = U*S

svd_data = US @ T

svd_data_scaled = scaler.inverse_transform(svd_data)

Below is the cell with commented code on plotting all these spectrograms :3

In [12]:

#fig, axs = plt.subplots(100, 2, figsize = (30, 200))

#for idx, ax in enumerate(axs.flat):
    
#    draw_img(ax, matrix[idx])

#plt.tight_layout()
#plt.show()

(342, 342) (342,) (342, 84741)


The function defined below takes in a data matrix and certain arguments that specify the extent to which it will execute principal component denoising.
Maybe it would be wise to implement a version of this function that derives the height and width parameters automatically.... Just something to think about.

In [ ]:
def feature_column_sub(data, feature_percentile = 50, observation_percentile = 50, num_components = 3, variance_explained = None, height = 141, width = 601):

    # Using Singular Value Decomposition to generate the principle eigenvectors

    U, S, T = np.linalg.svd(data, full_matrices=False)

    US = U*S

    # Filtering the specified number of principal components
    
    # First check if user wants to specify the number of components or if they will opt to calculate the number of components based on a variance threshold

    if variance_explained == None:

        # Creating a matrix to contain the pixel magnitude adjusted principal components

        signal_enhanced_features = np.zeros_like(T[0:num_components,:])


        # Now let i be the number of rows in the new Principal Component matrix to iterate through each "eigen - signal"

        for i in range(len(signal_enhanced_features)):
            
            # pick out the ith "eigen - signal" as a 2d spectrogram matrix

            feature = np.copy(T[i].reshape((height, width)))

            # Let j be the number of columns in the feature and iterate through every column, extract the user-specified percentile value and subtract this value from the whole column vector; move on to the next column, extract percentile value, subtract, iterate and repeat;
            # note that we set all values that become negative as a result to 0 automatically

            for j in range(feature.shape[1]):
                column = feature[:, j]
                percentile_value = np.percentile(column, feature_percentile)
                feature[:, j] = column - percentile_value
                feature[:, j][feature[:, j] < 0] = 0

            # Store this noise - adjusted "eigen - signal" in the new "Principal Component" matrix

            signal_enhanced_features[i] = feature.flatten()

    else:

        # Using the user's threshold for variance explained in the data, we need to calculate how many components will be kept during the PCA algorithm

        
        # First create a new whose values correspond to the variance explained for each singular value then create another vector whose entries are the ordered cumulative sum (until ~1) of those variance values

        # Then just add 1 to the index of the entry in this vector whose value is at least that of the user-defined threshold
        
        variance_individual = (S ** 2) / np.sum(S ** 2)

        cumulative_variance = np.cumsum(variance_individual)

        num_components = np.argmax(cumulative_variance >= variance_explained) + 1
        


        # Same code as above in the "if" block just using the new number of components

        signal_enhanced_features = np.zeros_like(T[0:num_components,:])

        for i in range(len(signal_enhanced_features)):

            feature = np.copy(T[i].reshape((height, width)))

            for j in range(feature.shape[1]):
                column = feature[:, j]
                percentile_value = np.percentile(column, feature_percentile)
                feature[:, j] = column - percentile_value
                feature[:, j][feature[:, j] < 0] = 0

            signal_enhanced_features[i] = feature.flatten()




    # Finally, we reconstruct the data matrix with our observations using the filtered components and the mixing matrix US

    reconstruction = US[:, 0:num_components] @ signal_enhanced_features[0:num_components, :]


    return reconstruction